## Assigment 3


In [1]:
require 'rest-client'

def fetch (url, headers = {accept: "*/*"}, user = "", pass="")
    response = RestClient::Request.execute({
      method: :get,
      url: url.to_s,
      user: user,
      password: pass,
      headers: headers})
    
    return response
  
    rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response
  
    rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response
  
    rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
end


:fetch

In [329]:
require 'bio'
response = fetch('http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&id=At3g54340')
if response
  record = response.body
  sequence = Bio::EMBL.new(record).to_biosequence
  #$contains = 0
  sequence.features.each do |feature|
    if feature.feature == "exon"
      strand = feature.position.match(/complement/)
      exon_region = feature.position.match(/\(([^\(]+)\)/)[1]
      exon = Bio::Feature.new('exon',exon_region)
      exon.append(Bio::Feature::Qualifier.new('gene', 'At3g54340'))
      feature.qualifiers.each do |qualifier|
        exon_ID = qualifier.value.match(/(exon_id=((.*?)exon\d))/)
        puts exon_ID
        exon.append(Bio::Feature::Qualifier.new('exon_ID', exon_ID[2]))
        if not strand.nil?
          exon.append(Bio::Feature::Qualifier.new('strand', '-'))
          exon_region_limit1 = exon_region.split('..')[0]
          exon_region_limit2 = exon_region.split('..')[1]
          exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i).complement
          exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
          exon.qualifiers.each do |x|
            if x.qualifier == "sequence" and not x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}.empty?
              match_positions = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}
              match_begin = match_positions.map(&:succ)
              match_end = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.end(0)}
              i = 0
              all_match = Array.new
              while i < match_begin.length
                match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                match.append(Bio::Feature::Qualifier.new('sequid', 'some_gene'))
                match.append(Bio::Feature::Qualifier.new('source', 'BioRuby'))
                match.append(Bio::Feature::Qualifier.new('type', 'exon'))                
                match.append(Bio::Feature::Qualifier.new('score', '.'))
                match.append(Bio::Feature::Qualifier.new('phase', '.'))
                match.append(Bio::Feature::Qualifier.new('attribute1', 'repetitive_motif=CTTCTT'))
                match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID[2]))
                all_match << match
                i = i +1
                #$contains += 1
              end
              #write_output(all_match)
            end 
          end
        else
          exon.append(Bio::Feature::Qualifier.new('strand', '+'))
          exon_region_limit1 = exon_region.split('..')[0]
          exon_region_limit2 = exon_region.split('..')[1]
          exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i)
          exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
          exon.qualifiers.each do |x|
            if x.qualifier == "sequence" and not x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}.empty?
              match_positions = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}
              match_begin = match_positions.map(&:succ)
              match_end = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.end(0)}
              i = 0
              all_match = Array.new
              while i < match_begin.length
                match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                match.append(Bio::Feature::Qualifier.new('sequid', 'some_gene'))
                match.append(Bio::Feature::Qualifier.new('source', 'BioRuby'))
                match.append(Bio::Feature::Qualifier.new('type', 'exon'))               
                match.append(Bio::Feature::Qualifier.new('score', '.'))
                match.append(Bio::Feature::Qualifier.new('phase', '.'))
                match.append(Bio::Feature::Qualifier.new('attribute1', 'repetitive_motif=CTTCTT'))
                match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID[2]))
                all_match << match
                i = i +1 
                #$contains += 1
              end
              #write_output1(all_match)
            end 
          end
        end
      end   
    end
  end
  #if $contains == 0
  #write_no_repetition(some_gene)
end

exon_id=AT3G54340.1.exon1
exon_id=AT3G54340.1.exon6
exon_id=AT3G54340.1.exon2
exon_id=AT3G54340.1.exon4
exon_id=AT3G54340.1.exon5
exon_id=AT3G54340.1.exon7
exon_id=AT3G54340.1.exon3


[#<Bio::Feature:0x000055a702c3cb80 @feature="source", @position="1..2175", @qualifiers=[#<Bio::Feature::Qualifier:0x000055a702c3c798 @qualifier="organism", @value="Arabidopsis thaliana">, #<Bio::Feature::Qualifier:0x000055a702c3c608 @qualifier="db_xref", @value="taxon:3702">]>, #<Bio::Feature:0x000055a702c3c4f0 @feature="gene", @position="complement(1..2175)", @qualifiers=[#<Bio::Feature::Qualifier:0x000055a702c3c130 @qualifier="gene", @value="AT3G54340">, #<Bio::Feature::Qualifier:0x000055a702c3c018 @qualifier="locus_tag", @value="AP3">, #<Bio::Feature::Qualifier:0x000055a702c33da0 @qualifier="note", @value="Floral homeotic protein APETALA 3 [Source:UniProtKB/Swiss-Prot;Acc:P35632]">]>, #<Bio::Feature:0x000055a702c33d50 @feature="mRNA", @position="join(complement(1761..2175),complement(1593..1659),complement(1448..1509),complement(1264..1363),complement(1050..1091),complement(729..773),complement(1..483))", @qualifiers=[#<Bio::Feature::Qualifier:0x000055a702c33738 @qualifier="gene", @

In [325]:
a = "heeelloheelloohello".gsub(/e(e)+/).map{Regexp.last_match.end(0)}
#b = a.map(&:succ)
a.class
for i in a 
  puts i
end
a.length

4
10


2

In [ ]:
    case 
      when feature.feature == "gene"
      feature.qualifiers.each do |x|
        if x.qualifier == "note"
          source = x.value.match(/Source:(.*?);/)[1]
          if not source[1].nil?
            source = Bio::Feature::Qualifier.new('source', source)
          else
            source = Bio::Feature::Qualifier.new('source', 'unknown')
          end
        end
      end

In [341]:
def search_exons(some_gene)
    response = fetch("http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&id=#{some_gene}")
    if response
      record = response.body
      sequence = Bio::EMBL.new(record).to_biosequence
      $contains = 0
      sequence.features.each do |feature|
        puts feature.feature
        if feature.feature == "exon"
          strand = feature.position.match(/complement/)
          exon_region = feature.position.match(/\(([^\(]+)\)/)[1]
          exon = Bio::Feature.new('exon',exon_region)
          exon.append(Bio::Feature::Qualifier.new('gene', some_gene))
          feature.qualifiers.each do |qualifier|
            exon_ID = qualifier.value.match(/(exon)(\d)/)
            exon.append(Bio::Feature::Qualifier.new('number', exon_ID[2]))
            if strand.nil?
              exon.append(Bio::Feature::Qualifier.new('strand', '+'))
              exon_region_limit1 = exon_region.split('..')[0]
              exon_region_limit2 = exon_region.split('..')[1]
              exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i)
              exon.append(Bio::Feature::Qualifier.new('exon_sequence', exon_sequence))
              exon.qualifiers.each do |x|
                if x.qualifier == "sequence" and not x.value.gsub(/ctt(ctt)+/).map{Regexp.last_match.begin(0)}.empty?
                  match_positions = x.value.gsub(/ctt(ctt)+/).map{Regexp.last_match.begin(0)}
                  match_begin = match_positions.map(&:succ)
                  match_end = x.value.gsub(/ctt(ctt)+/).map{Regexp.last_match.end(0)}
                  i = 0
                  all_match = Array.new
                  while i < match_begin.length
                    match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                    match.append(Bio::Feature::Qualifier.new('sequid', some_gene))
                    match.append(Bio::Feature::Qualifier.new('strand', '+'))
                    match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID[2]))
                    all_match << match
                    i = i +1
                    $contains += 1
                    puts $contains
                  end
                  ##write_gff1(all_match)
                end
              end
            else
              exon.append(Bio::Feature::Qualifier.new('strand', '-'))
              exon_region_limit1 = exon_region.split('..')[0]
              exon_region_limit2 = exon_region.split('..')[1]
              exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i).complement
              exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
              exon.qualifiers.each do |x|
                if x.qualifier == "sequence" and not x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}.empty?
                  match_positions = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}
                  match_begin = match_positions.map(&:succ)
                  match_end = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.end(0)}
                  i = 0
                  all_match = Array.new
                  while i < match_begin.length
                    match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                    match.append(Bio::Feature::Qualifier.new('sequid', some_gene))
                    match.append(Bio::Feature::Qualifier.new('strand', '-'))
                    match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID[2]))
                    all_match << match
                    i = i +1
                    $contains += 1
                    puts $contains
                    end
                  ##write_gff1(all_match)
                end
              end
            end
          end
        end
      end
      if $contains == 0
        #write_no_cttctt(some_gene)
        puts $contains
      end
    end
  end

:search_exons

In [342]:
search_exons('AT3G54340')

source
gene
mRNA
CDS
exon
1
2
exon
exon
exon
3
exon
exon
exon
mRNA
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
variation
var